In [9]:
import pandas as pd

test = pd.read_csv("data/test_pos.csv")
print(test.shape )
test.head()



(5991, 3)


,persian,english,persian_pos
0,و به زیر لیوانی کاغذی من نگاه کن,and look down at my embossed paper coaster,"CCONJ ADP NOUN,EZ NOUN ADJ,EZ PRON NOUN VERB"
1,تو برو یه همچین کاری بکن,you go and do something like this,PRON VERB NUM NOUN NOUN VERB
2,پذیرش,adopcje,NOUN
3,مدیامکس اخبار قره باغ کوهستانی وزیر خارجه ارمن...,mediamax news nagorno karabakh armenian fm bak...,"NOUN NOUN,EZ NOUN,EZ ADJ,EZ NOUN,EZ ADJ,EZ NOU..."
4,صد خانه در آن واحد متعارف و ترسناک در زیر آسما...,a hundred houses at once conventional and grot...,"NUM NOUN ADP NOUN,EZ NOUN,EZ ADJ CCONJ ADJ ADP..."


In [10]:
train=pd.read_csv("data/train_pos.csv")
print(train.shape )
train.head()

(387374, 3)


,persian,english,persian_pos
0,توافقنامه با دانشگاه دولتی فرهنگ مسکو در مورد ...,the agreements with the moscow state universit...,"NOUN ADP NOUN,EZ ADJ,EZ NOUN,EZ NOUN ADP NOUN,..."
1,دوره های ویدیویی اسپانیایی برای مبتدیان,spanish video courses for beginners,"NOUN,EZ ADJ,EZ ADJ ADP,EZ NOUN"
2,ترجیح می دم مرده بودم,i d rather be dead,NOUN VERB VERB
3,بنابراین دولت در سنت شرکتی به وضوح مداخله گر و...,the state in the corporatist tradition is thus...,"SCONJ NOUN ADP NOUN,EZ NOUN ADP NOUN,EZ ADJ CC..."
4,این مطالعه همچنین نشان داد که زنان مطلقه بیشتر...,the study also found that divorced women are m...,"DET NOUN CCONJ NOUN VERB SCONJ NOUN,EZ ADJ,EZ ..."


In [11]:
validation = pd.read_csv("data/validation_pos.csv")
print(train.shape )
train.head()

(387374, 3)


,persian,english,persian_pos
0,توافقنامه با دانشگاه دولتی فرهنگ مسکو در مورد ...,the agreements with the moscow state universit...,"NOUN ADP NOUN,EZ ADJ,EZ NOUN,EZ NOUN ADP NOUN,..."
1,دوره های ویدیویی اسپانیایی برای مبتدیان,spanish video courses for beginners,"NOUN,EZ ADJ,EZ ADJ ADP,EZ NOUN"
2,ترجیح می دم مرده بودم,i d rather be dead,NOUN VERB VERB
3,بنابراین دولت در سنت شرکتی به وضوح مداخله گر و...,the state in the corporatist tradition is thus...,"SCONJ NOUN ADP NOUN,EZ NOUN ADP NOUN,EZ ADJ CC..."
4,این مطالعه همچنین نشان داد که زنان مطلقه بیشتر...,the study also found that divorced women are m...,"DET NOUN CCONJ NOUN VERB SCONJ NOUN,EZ ADJ,EZ ..."


In [3]:
from transformers import pipeline

ner_fa = pipeline(
    "token-classification",
    model="HooshvareLab/bert-fa-base-uncased-ner-peyma",
    tokenizer="HooshvareLab/bert-fa-base-uncased-ner-peyma",
    aggregation_strategy="simple"
)

2025-09-28 19:47:36.085232: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-28 19:47:36.148641: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-28 19:47:37.517856: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased-ner-pe

In [1]:
from datasets import load_dataset
import json

# --- Load CSV (expects a column named "persian") ---
ds = load_dataset("csv", data_files="data/validation_pos.csv", split="train")




Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
# -*- coding: utf-8 -*-
# Full script: Persian NER → per-token tags → save as last column

import re
import json


# --- NER batch function: emits per-token tags like "0 0 0 LOC 0" ---
def run_ner(batch):
    texts = batch["persian"]                  # list[str]
    ents_list = ner_fa(texts)                 # list[list[dict]] ; aggregation_strategy="simple"

    def norm_label(lbl: str) -> str:
        if not lbl:
            return "0"
        s = str(lbl).strip()
        if s[:2] in ("B_", "I_", "B-", "I-"):
            s = s[2:]
        s = s.upper()
        alias = {
            "PER": "PERSON", "PERS": "PERSON", "PERSON": "PERSON",
            "ORG": "ORG", "ORGANIZATION": "ORG",
            "LOC": "LOCATION", "LOCATION": "LOCATION", "GPE": "LOCATION",
            "FAC": "FACILITY", "FACILITY": "FACILITY",
            "DAT": "DATE", "DATE": "DATE",
            "TIM": "TIME", "TIME": "TIME",
            "MON": "MONEY", "MONEY": "MONEY",
            "PCT": "PERCENT", "PERCENT": "PERCENT",
            "QUANTITY": "QUANTITY", "CARDINAL": "CARDINAL", "ORDINAL": "ORDINAL",
            "MISC": "MISC", "EVENT": "EVENT", "PRODUCT": "PRODUCT",
            "WORK_OF_ART": "WORK_OF_ART", "LAW": "LAW", "LANGUAGE": "LANGUAGE",
            "NORP": "NORP",
        }
        return alias.get(s, s)

    def tag_sequence(text: str, ents: list[dict]) -> str:
        """
        Tokenize by whitespace (runs of non-space chars) and align labels by char spans.
        Non-entity tokens get "0". If a token overlaps any entity span, it gets that entity's label.
        """
        # 1) whitespace tokens with char spans
        tokens = [(m.group(0), m.start(), m.end()) for m in re.finditer(r"\S+", text or "")]
        # 2) entity spans (start,end,label)
        spans = []
        for e in ents or []:
            lbl = e.get("entity_group") or e.get("entity") or ""
            start = e.get("start")
            end = e.get("end")
            if lbl and start is not None and end is not None:
                spans.append((int(start), int(end), norm_label(lbl)))
        # 3) assign label per token (simple overlap rule)
        tags = []
        for _, t0, t1 in tokens:
            tag = "0"
            for s0, s1, lab in spans:
                if t0 < s1 and t1 > s0:  # overlap
                    tag = lab
                    break
            tags.append(tag)
        return " ".join(tags)

    # Build outputs
    seqs = []
    for text, ents in zip(texts, ents_list):
        seqs.append(tag_sequence(text, ents))

    batch["fa_ner_seq"] = seqs   # e.g., "0 0 0 LOC 0"
    return batch


# ---- Map over dataset ----
# NOTE: assumes you already have `ds` and `ner_fa`.
ds_out = ds.map(run_ner, batched=True, batch_size=32)

# ---- Save with fa_ner_seq as the LAST column ----
# We’ll keep original column order and append the new column at the end.
orig_cols = ds.column_names
wanted_order = orig_cols + ["fa_ner_seq"]
wanted_order = [c for c in wanted_order if c in ds_out.column_names]  # robust if columns vary

# Option A (simple & explicit control via pandas)
df = ds_out.to_pandas()
df = df[wanted_order]
df.to_csv("data/validation_pos_ner.csv", index=False)
print("Saved validation_pos_ner.csv with 'fa_ner_seq' appended as the last column.")

# --- Optional: if your datasets version supports specifying columns directly, you can use:
# ds_out.to_csv("train_ner.csv", columns=wanted_order)


Map:   0%|          | 0/5990 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Saved validation_pos_ner.csv with 'fa_ner_seq' appended as the last column.
